In [1]:
import os
os.chdir('../')
%pwd

'd:\\Programming\\DataScienceProjects\\mlflow-project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_file: Path

In [5]:
from mlflowProject.constants import *
from mlflowProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath =  PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self):
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_file = config.data_file,
        )

        return data_transformation_config

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from mlflowProject import logger

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # different transformation techniques can be added
    def train_test_splitting(self):
        df = pd.read_csv(self.config.data_file)
        train, test = train_test_split(df, test_size=0.2, random_state=42)

        train_data_path = os.path.join(self.config.root_dir, 'train.csv')
        test_data_path = os.path.join(self.config.root_dir, 'test.csv')

        train.to_csv(train_data_path, index=False)
        test.to_csv(test_data_path, index=False)

        logger.info(f"train data shape {train.shape}")
        logger.info(f"test data shape {test.shape}")

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    logger.error(e)
    raise e

[2024-01-20 00:31:44,939: INFO: common] File loaded: config\config.yaml
[2024-01-20 00:31:44,943: INFO: common] File loaded: params.yaml
[2024-01-20 00:31:44,950: INFO: common] File loaded: schema.yaml
[2024-01-20 00:31:44,950: INFO: common] Directory already exists: artifacts
[2024-01-20 00:31:44,950: INFO: common] Directory already exists: artifacts/data_transformation
[2024-01-20 00:31:45,060: INFO: 2423780382] train data shape (1279, 12)
[2024-01-20 00:31:45,060: INFO: 2423780382] test data shape (320, 12)
